In [2]:
import xlrd
import sys
import csv
import pickle
import numpy as np
import copy
from matplotlib import pyplot as plt
import math
import random
import pandas as pd
from numpy.random import choice
from scipy import stats
from collections import defaultdict
from operator import itemgetter, sub
from sklearn.metrics import roc_curve, auc, accuracy_score, explained_variance_score, mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
class tempDebugging:
    'temporary functions for debugging and detecting the system, which will not be inlcuded in main function'
    @staticmethod
    def doubleLinkPercentage(df):
        'return the number of links that is doulbe direction'
        linkNum = 0
        doubleDirectNum = 0
        for curCell in df.index:
            nbList = df.loc[curCell,'Cluster'] + [] # to make it a copy instead of work on the same list
            linkNum += len(nbList)
            for nb in nbList:
                if curCell in df.loc[nb,'Cluster']:
                    doubleDirectNum += 1
        print(linkNum)
        print(doubleDirectNum)
        print((doubleDirectNum+0.0)/linkNum)
        return linkNum, doubleDirectNum

#tempDebugging.doubleLinkPercentage(df)

In [4]:
class InitData:
    'functions to deal with input'
    @staticmethod
    def genInput(filename):
        'generate DataFrame input from .csv file'
        data = pd.read_csv(filename)
        data = data.loc[:,['Iteration Num','CellId','PilotPower','Load','TBS(kbit)','Users',\
                          'Cluster','DeltaLoad']]
        tempIterationNum = data['Iteration Num'].iloc[0]
        data =data[data['Iteration Num'] == tempIterationNum ]
        return data    
    
    @staticmethod
    def init(originDf):
        'convert Cluster and DeltaLoad to list'
        rowNum = len(originDf)
        newCluster =[]
        for rowIter in range(0,rowNum):
            nb = originDf.iloc[rowIter]['Cluster']
            nb = nb.split(';')
            nb = nb[0:-1]
            nb =[int(j) for j in nb]
            newCluster.append(nb)  
        newCluster = pd.Series(newCluster,originDf.index)
        originDf['Cluster'] = newCluster
        #processing DeltaLoad
        rowNum = len(originDf)
        newCluster =[]
        for rowIter in range(0,rowNum):
            nb = originDf.iloc[rowIter]['DeltaLoad']
            nb = nb.split(';')
            nb = nb[0:-1]
            nb =[float(j) for j in nb]
            newCluster.append(nb)  
        newCluster = pd.Series(newCluster,originDf.index)
        originDf['DeltaLoad'] = newCluster
        return
    
    @staticmethod
    def getFeatureIndex(df):
        'to add the AverageLoad feature, AffectCluster feature and set cellID as index'
        # averageLoad feature
        rowNum = len(df)
        newFeature = []
        for rowIter in range(0,rowNum):
            averageLoad = np.mean(df.iloc[rowIter]['DeltaLoad'])
            averageLoad = averageLoad*5.0/6.0
            averageLoad = df.iloc[rowIter]['Load'] - averageLoad
            newFeature.append(averageLoad)
        newFeature = pd.Series(newFeature,df.index)
        df['averageLoad'] = newFeature
        # set new index now
        df.set_index('CellId',inplace = True)
        # AffectCluster feature
        rowNum = len(df)
        AffectCluster =[set() for x in range(rowNum)]
        AffectCluster = pd.Series(AffectCluster, df.index)
        df['affectCluster'] = AffectCluster
        for startCell in df.index:
            for endCell in df.loc[startCell,'Cluster']:
                df.loc[endCell,'affectCluster'].add(startCell)        
        return

In [5]:
class Config:
    'configure the parameters of the system'
    def __init__(self):
        self.optIterNum = 20
        self.actDomain = list(np.linspace(30,36,num = 7))
        self.T = 0.5
        self.result_TtooSmall = 0

In [6]:
class TrainingNNModel:
    'to train neural network for the features'
    def __init__(self):
        self.model = MLPRegressor() # neural network model
        self.scaler = StandardScaler() # scaler of input
        
    def train(self,feature,target):
        self.scaler.fit(feature)
        self.model.fit(self.scaler.transform(feature),target)
        print(self.model.score(self.scaler.transform(feature),target))
        
    def predict(self,feature):
        return self.model.predict(self.scaler.transform(feature))        

In [7]:
class ProcessFuns:
    'functions in processing'    
    @staticmethod
    def getFeature(tempCell, df, changeCell=-1, action=-1):
        'feature as [Pilot power, neighbor pilot power *5, averageLoad]'
        feature = []
        # deal with the case without changeCell
        if changeCell == -1:
            changeCell = tempCell
            action = df.loc[tempCell,'PilotPower']         
        # now process
        if tempCell != changeCell:
            feature.append(df.loc[tempCell,'PilotPower'])
        else:
            feature.append(action)
        for nb in df.loc[tempCell,'Cluster']:
            if nb != changeCell:
                feature.append(df.loc[nb,'PilotPower'])
            else:
                feature.append(action)
        while len(feature)<6:
            feature.append(29.5) # to be revised
        feature.append(df.loc[tempCell,'averageLoad'])
        return feature
    
    @staticmethod
    def probGen(nnModel,changeCell,df,config):
        cellList = [changeCell] + list(df.loc[changeCell, 'affectCluster'])
        probWeight = []
        for act in config.actDomain:
            value = []
            for tempCell in cellList:
                tempFeature = ProcessFuns.getFeature(tempCell,df,changeCell,act)
                value.append(nnModel.predict([tempFeature])[0])
            probWeight.append(sum(value))
        if config.T == 0:
            prob = [0.0] * len(probWeight)
            maxInd = probWeight.index(max(probWeight))
            prob[maxInd] = 1.0
        elif max(probWeight)/config.T >= math.log(sys.float_info.max):
            config.result_TtooSmall  = 1
            prob = [0.0] * len(probWeight)
            maxInd = probWeight.index(max(probWeight))
            prob[maxInd] = 1.0
        else:
            probWeight = [math.exp(x/config.T) for x in probWeight]
            prob = [x/sum(probWeight) for x in probWeight]
        return prob

In [12]:
filename = 's.csv'
# initialize data
myInput = InitData.genInput(filename)# to change input here
df = myInput.copy() 
InitData.init(df)
InitData.getFeatureIndex(df)

In [9]:
# config settings
config = Config()
config.optIterNum = 20
config.actDomain = list(np.linspace(30,36,num = 13)) # num = 7 or 13
config.T = 0 

In [9]:
# --------now, proceed to proceeding ----------------
nnModel = pickle.load(open('s.csv.pkl','rb'))
utility = []
for curIter in range(0,config.optIterNum):
    curUtility = 0
    for curCell in df.index:
        tempFeature = ProcessFuns.getFeature(curCell,df)
        curUtility += nnModel.predict([tempFeature])[0]
    utility.append(curUtility)
    for curCell in df.index:
        prob = ProcessFuns.probGen(nnModel, curCell, df, config)
        curAct = choice(config.actDomain, 1, p=prob)
        df.loc[curCell,'PilotPower'] = curAct   
print(utility)

[34978.821132349891, 36783.626968747369, 37079.335864770721, 37164.503721260022, 37178.792162100421, 37197.967324014258, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606, 37199.449222374606]


In [10]:
myOutput = df['PilotPower'].copy()
#myOutput is the final output

In [16]:
myInput.head(3)
myInput.loc[0,'DeltaLoad']

'-0.3063;-0.305;0.1875;-0.306;0.2616;'